# 1. Make data phase

Make data by yoursefl using camera

In [ ]:
# Run this if you thing you have library :v
import cv2
import mediapipe as mp
import pandas as pd

In [ ]:
# Else run this if you thing you don't have
%pip install opencv-python
%pip install mediapipe
%pip install pandas

import cv2
import mediapipe as mp
import pandas as pd

### Read infomation from camera

In [ ]:
cap = cv2.VideoCapture(0)
# If got error, try to replace index 0 by 1, 2,...

### Using mediapipe lib

In [ ]:
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils

### Initialize variables

In [ ]:
lm_list = []           # List of frame's landmarks
labels = "HEADSWING"   # Name of the movement which you take data 
no_of_frames = 600     # Frames you take data

## Function 

### Function to extract frame's landmarks and storage in lm_list

In [ ]:
def make_landmark_timestep(results):
    print(results.pose_landmarks.landmark)
    c_lm = []
    for id, lm in enumerate(results.pose_landmarks.landmark):
        c_lm.append(lm.x)
        c_lm.append(lm.y)
        c_lm.append(lm.z)
        c_lm.append(lm.visibility)
    return c_lm

### Function draw landmark on the image

In [ ]:
def draw_landmark_on_image(mpDraw, results, img):

    # Draw lines
    mpDraw.draw_landmarks(img, results.pose_landmarks, mpPose.POSE_CONNECTIONS)

    # Draw nodes
    for id, lm in enumerate(results.pose_landmarks.landmark):
        h, w, c = img.shape
        print(id, lm)
        cx, cy = int(lm.x * w), int(lm.y * h)
        cv2.circle(img, (cx, cy), 10, (0, 0, 255), cv2.FILLED)
    return img

In [ ]:
while len(lm_list) <= no_of_frames:
    ret, frame = cap.read()
    if ret:

        # Pose recognition
        frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frameRGB)
        if results.pose_landmarks:

            # Append skeleton's paramaters to lm_list
            lm = make_landmark_timestep(results)
            lm_list.append(lm)

            # Draw skeleton on image
            frame = draw_landmark_on_image(mpDraw, results, frame)
        cv2.imshow('image', frame)
        if cv2.waitKey(1) == ord("q"):   # If press Q button, end the window
            break

# Write data to csv file     
df = pd.DataFrame(lm_list)
df.to_csv(labels + ".txt")
cap.release()
cv2.destroyAllWindows()

# 2. Training Phase

In [ ]:
# Run this if you think you have library :v
import numpy as np
import pandas as pd

from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential

from sklearn.model_selection import train_test_split

In [ ]:
# Else run this if you think you don't have
%pip install numpy
%pip install pandas
%pip install tensorflow
%pip install keras
%pip install scikit-learn

In [ ]:
X = []  
Y = []
data_name = ["HEADSWING.txt", "HANDSWING.txt", "BODYSWING.txt"]   # Training CSV files
no_of_timestep = 10 

### Turn each 10 data frames (timestep) into 1 data point

In [ ]:
for idx in range(3):
    data = pd.read_csv(data_name[idx])
    dataset = data.iloc[:, 1:].values
    n_sample = len(dataset)
    for i in range(no_of_timestep, n_sample):
        X.append(dataset[i - no_of_timestep:i, :])  # Lấy 10 timestep liên tiếp
        Y.append(idx)
        # 0 là Headswing
        # 1 là Handswing
        # 2 là Bodyswing

### Convert X, y to numpy arrray and divide to train, validate, test dataset

In [ ]:
X, y = np.array(X), np.array(Y)
print(X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### Build model

In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dense(units=3, activation='softmax'))

### View model

In [ ]:
model.summary()

### Compile Model

In [ ]:
model.compile(optimizer="adam", metrics=['accuracy'], loss="sparse_categorical_crossentropy")

### Fit model

In [ ]:
model.fit(X_train, y_train, epochs=16, batch_size=32, validation_data=(X_test, y_test))
model.save('model_multi.h5')

# 3. Inference Model

In [ ]:
# Run this if you think you have library :v
import cv2
import keras
import mediapipe as mp
import numpy as np
import pandas as pd
import threading

In [ ]:
# Else run this if you think you don't have
%pip install opencv-python
%pip install tensorflow
%pip install keras
%pip install numpy
%pip install pandas

In [ ]:
# Read frame from Camera
cap = cv2.VideoCapture(0)

In [ ]:
# Initalize Mediapipe
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils

In [ ]:
lm_list = []
no_of_frames = 600

i = 0
warmup_frame = 60

In [ ]:
# Load model
model = keras.models.load_model("model_multi.h5")

## Functions

### Draw landmarks on image

In [ ]:
def draw_landmark_on_image(mpDraw, results, img):
    # Draw lines
    mpDraw.draw_landmarks(img, results.pose_landmarks, mpPose.POSE_CONNECTIONS)

    # Draw nodes
    for id, lm in enumerate(results.pose_landmarks.landmark):
        h, w, c = img.shape
        # print(id, lm)
        cx, cy = int(lm.x * w), int(lm.y * h)
        cv2.circle(img, (cx, cy), 10, (0, 0, 255), cv2.FILLED)
    return img


### Make timestep's landmark

In [ ]:
def make_landmark_timestep(results):
    # print(results.pose_landmarks.landmark)
    c_lm = []
    for id, lm in enumerate(results.pose_landmarks.landmark):
        c_lm.append(lm.x)
        c_lm.append(lm.y)
        c_lm.append(lm.z)
        c_lm.append(lm.visibility)
    return c_lm

### Draw class on image

In [ ]:
def draw_class_on_image(label, img):
    font = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10, 30)
    fontScale = 1
    fontColor = (0, 255, 0)
    thickness = 2
    lineType = 2
    cv2.putText(img, label,
                bottomLeftCornerOfText,
                font,
                fontScale,
                fontColor,
                thickness,
                lineType)
    return img

### From the result, detect the label

In [ ]:
def detect(model, lm_list):
    global label
    lm_list = np.array(lm_list)
    lm_list = np.expand_dims(lm_list, axis=0)
    print(lm_list.shape)
    results = model.predict(lm_list)
    print(results.shape)
    lb_idx = np.argmax(results, axis=1)
    if lb_idx == 0:
        label = "Headswing"
    elif lb_idx == 1:
        label = "Handswing"
    elif lb_idx == 2:
        label = "Bodyswing"
    return label

## Main function

In [ ]:
label = "Warmup...."
while len(lm_list) <= no_of_frames:
    ret, frame = cap.read()
    if ret:

        # Nhận diện pose
        frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frameRGB)
        i = i + 1
        if i > warmup_frame:
            print("Start Detect...")
            if results.pose_landmarks:

                # Ghi nhận thông số khung xương
                lm = make_landmark_timestep(results)
                lm_list.append(lm)
                if len(lm_list) == 10:
                    # Đưa vào model nhận diện
                    t1 = threading.Thread(target=detect, args=(model, lm_list,))
                    t1.start()
                    lm_list = []
                # Vẽ khung xương lên ảnh
                frame = draw_landmark_on_image(mpDraw, results, frame)
        frame = draw_class_on_image(label, frame)
        cv2.imshow('image', frame)
        if cv2.waitKey(1) == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()